In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Esophageal_Cancer"
cohort = "GSE100843"

# Input paths
in_trait_dir = "../../input/GEO/Esophageal_Cancer"
in_cohort_dir = "../../input/GEO/Esophageal_Cancer/GSE100843"

# Output paths
out_data_file = "../../output/preprocess/Esophageal_Cancer/GSE100843.csv"
out_gene_data_file = "../../output/preprocess/Esophageal_Cancer/gene_data/GSE100843.csv"
out_clinical_data_file = "../../output/preprocess/Esophageal_Cancer/clinical_data/GSE100843.csv"
json_path = "../../output/preprocess/Esophageal_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Expression data from nonrandomized trial of vitamin D in Barrett's esophagus"
!Series_summary	"Vitamin D deficiency has been associated with increased esophageal cancer risk.  Vitamin D controls many downstream regulators of cellular processes including proliferation, apoptosis, and differentiation.  We evaluated the effects of vitamin D supplementation on global gene expression in patients with Barrett's esophagus."
!Series_summary	"We used microarrays to assess global gene expression in Barrett's esophagus patients who received vitamin D supplementation."
!Series_overall_design	"Patients in Arm A with Barrett's esophagus with high grade dysplasia were given vitamin D3 50,000 IU weekly for 2 weeks.  Patients in Arm B with Barrett's esophagus with low grade dysplasia or no dysplasia were given vitamin D3 50,000 IU weekly for 12 weeks.  In both arms, biopsies were obtained from two sites: Barrett's esophagus segment (IM) and normal squamous mucosa 

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import os
import json
import pandas as pd
from typing import Callable, Optional, Dict, Any

# Step 1: Determine gene expression data availability
# From the background information, we can see they used microarrays for gene expression in Barrett's esophagus patients
is_gene_available = True  # Microarray data typically contains gene expression data

# Step 2: Identify row numbers for trait, age, and gender
# From sample characteristics, we see information about tissue, arm, and timepoint

# For trait data, we can use the 'tissue' information as it tells us whether it's Barrett's esophagus or normal tissue
trait_row = 0  # The tissue information is in row 0

# Age and gender are not available in the sample characteristics
age_row = None
gender_row = None

# Step 2.2: Define conversion functions
def convert_trait(value_str):
    """Convert tissue information to binary trait value (1 for Barrett's esophagus, 0 for normal)"""
    if not isinstance(value_str, str):
        return None
    
    # Extract the value after the colon
    if ":" in value_str:
        value = value_str.split(":", 1)[1].strip()
    else:
        value = value_str.strip()
    
    if "Barrett's esophagus" in value:
        return 1  # Barrett's esophagus (pathological condition)
    elif "Normal" in value:
        return 0  # Normal esophageal squamous mucosa
    else:
        return None

def convert_age(value_str):
    """Placeholder function as age data is not available"""
    return None

def convert_gender(value_str):
    """Placeholder function as gender data is not available"""
    return None

# Step 3: Save metadata about dataset usability
is_trait_available = trait_row is not None

validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# Step 4: Extract clinical features if trait_row is not None
if trait_row is not None:
    # Create a DataFrame from the sample characteristics dictionary provided in the previous step
    # The dictionary has row numbers as keys and lists of characteristic values as values
    sample_chars = {
        0: ["tissue: Barrett's esophagus segment", 'tissue: Normal esophageal squamous mucosa'], 
        1: ['arm: Arm A', 'arm: Arm B'], 
        2: ['timepoint (t0=before, t1=after): T0', 'timepoint (t0=before, t1=after): T1']
    }
    
    try:
        # Convert the dictionary to a format suitable for geo_select_clinical_features
        # Each row in the dictionary becomes a column in the DataFrame
        clinical_data = pd.DataFrame()
        for key, values in sample_chars.items():
            # Create a series for each characteristic
            clinical_data[key] = values
        
        # Extract clinical features using the library function
        selected_clinical_df = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        # Preview the extracted clinical data
        preview = preview_df(selected_clinical_df)
        print("Preview of extracted clinical data:")
        print(preview)
        
        # Create the output directory if it doesn't exist
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        
        # Save the extracted clinical data
        selected_clinical_df.to_csv(out_clinical_data_file, index=False)
        print(f"Clinical data saved to {out_clinical_data_file}")
    except Exception as e:
        print(f"Error processing clinical data: {e}")
else:
    print("Trait data is not available. Skipping clinical feature extraction.")


Preview of extracted clinical data:
{0: [1.0], 1: [nan], 2: [nan]}
Clinical data saved to ../../output/preprocess/Esophageal_Cancer/clinical_data/GSE100843.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. First, let's examine the structure of the matrix file to understand its format
import gzip

# Peek at the first few lines of the file to understand its structure
with gzip.open(matrix_file, 'rt') as file:
    # Read first 100 lines to find the header structure
    for i, line in enumerate(file):
        if '!series_matrix_table_begin' in line:
            print(f"Found data marker at line {i}")
            # Read the next line which should be the header
            header_line = next(file)
            print(f"Header line: {header_line.strip()}")
            # And the first data line
            first_data_line = next(file)
            print(f"First data line: {first_data_line.strip()}")
            break
        if i > 100:  # Limit search to first 100 lines
            print("Matrix table marker not found in first 100 lines")
            break

# 3. Now try to get the genetic data with better error handling
try:
    gene_data = get_genetic_data(matrix_file)
    print(gene_data.index[:20])
except KeyError as e:
    print(f"KeyError: {e}")
    
    # Alternative approach: manually extract the data
    print("\nTrying alternative approach to read the gene data:")
    with gzip.open(matrix_file, 'rt') as file:
        # Find the start of the data
        for line in file:
            if '!series_matrix_table_begin' in line:
                break
                
        # Read the headers and data
        import pandas as pd
        df = pd.read_csv(file, sep='\t', index_col=0)
        print(f"Column names: {df.columns[:5]}")
        print(f"First 20 row IDs: {df.index[:20]}")
        gene_data = df


Found data marker at line 64
Header line: "ID_REF"	"GSM2694849"	"GSM2694850"	"GSM2694851"	"GSM2694852"	"GSM2694853"	"GSM2694854"	"GSM2694855"	"GSM2694856"	"GSM2694857"	"GSM2694858"	"GSM2694859"	"GSM2694860"	"GSM2694861"	"GSM2694862"	"GSM2694863"	"GSM2694864"	"GSM2694865"	"GSM2694866"	"GSM2694867"	"GSM2694868"	"GSM2694869"	"GSM2694870"	"GSM2694871"	"GSM2694872"	"GSM2694873"	"GSM2694874"	"GSM2694875"	"GSM2694876"	"GSM2694877"	"GSM2694878"	"GSM2694879"	"GSM2694880"	"GSM2694881"	"GSM2694882"	"GSM2694883"	"GSM2694884"	"GSM2694885"	"GSM2694886"	"GSM2694887"	"GSM2694888"	"GSM2694889"	"GSM2694890"	"GSM2694891"	"GSM2694892"	"GSM2694893"	"GSM2694894"	"GSM2694895"	"GSM2694896"	"GSM2694897"	"GSM2694898"	"GSM2694899"	"GSM2694900"	"GSM2694901"	"GSM2694902"	"GSM2694903"	"GSM2694904"	"GSM2694905"	"GSM2694906"	"GSM2694907"	"GSM2694908"	"GSM2694909"	"GSM2694910"	"GSM2694911"	"GSM2694912"	"GSM2694913"	"GSM2694914"	"GSM2694915"	"GSM2694916"	"GSM2694917"	"GSM2694918"	"GSM2694919"	"GSM2694920"	"GSM2694921"	

Index(['7892501', '7892502', '7892503', '7892504', '7892505', '7892506',
       '7892507', '7892508', '7892509', '7892510', '7892511', '7892512',
       '7892513', '7892514', '7892515', '7892516', '7892517', '7892518',
       '7892519', '7892520'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Examine the identifiers in the gene expression data
# Based on the preview, we see identifiers like 7892501, 7892502, etc.
# These are numeric identifiers, not standard human gene symbols (which would be like BRCA1, TP53, etc.)
# These appear to be probe IDs from a microarray platform that need to be mapped to gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Let's first examine the structure of the SOFT file before trying to parse it
import gzip

# Look at the first few lines of the SOFT file to understand its structure
print("Examining SOFT file structure:")
try:
    with gzip.open(soft_file, 'rt') as file:
        # Read first 20 lines to understand the file structure
        for i, line in enumerate(file):
            if i < 20:
                print(f"Line {i}: {line.strip()}")
            else:
                break
except Exception as e:
    print(f"Error reading SOFT file: {e}")

# 2. Now let's try a more robust approach to extract the gene annotation
# Instead of using the library function which failed, we'll implement a custom approach
try:
    # First, look for the platform section which contains gene annotation
    platform_data = []
    with gzip.open(soft_file, 'rt') as file:
        in_platform_section = False
        for line in file:
            if line.startswith('^PLATFORM'):
                in_platform_section = True
                continue
            if in_platform_section and line.startswith('!platform_table_begin'):
                # Next line should be the header
                header = next(file).strip()
                platform_data.append(header)
                # Read until the end of the platform table
                for table_line in file:
                    if table_line.startswith('!platform_table_end'):
                        break
                    platform_data.append(table_line.strip())
                break
    
    # If we found platform data, convert it to a DataFrame
    if platform_data:
        import pandas as pd
        import io
        platform_text = '\n'.join(platform_data)
        gene_annotation = pd.read_csv(io.StringIO(platform_text), delimiter='\t', 
                                      low_memory=False, on_bad_lines='skip')
        print("\nGene annotation preview:")
        print(preview_df(gene_annotation))
    else:
        print("Could not find platform table in SOFT file")
        
        # Try an alternative approach - extract mapping from other sections
        with gzip.open(soft_file, 'rt') as file:
            for line in file:
                if 'ANNOTATION information' in line or 'annotation information' in line:
                    print(f"Found annotation information: {line.strip()}")
                if line.startswith('!Platform_title') or line.startswith('!platform_title'):
                    print(f"Platform title: {line.strip()}")
            
except Exception as e:
    print(f"Error processing gene annotation: {e}")


Examining SOFT file structure:
Line 0: ^DATABASE = GeoMiame
Line 1: !Database_name = Gene Expression Omnibus (GEO)
Line 2: !Database_institute = NCBI NLM NIH
Line 3: !Database_web_link = http://www.ncbi.nlm.nih.gov/geo
Line 4: !Database_email = geo@ncbi.nlm.nih.gov
Line 5: ^SERIES = GSE100843
Line 6: !Series_title = Expression data from nonrandomized trial of vitamin D in Barrett's esophagus
Line 7: !Series_geo_accession = GSE100843
Line 8: !Series_status = Public on Jul 06 2017
Line 9: !Series_submission_date = Jul 05 2017
Line 10: !Series_last_update_date = Jul 25 2021
Line 11: !Series_pubmed_id = 28922414
Line 12: !Series_summary = Vitamin D deficiency has been associated with increased esophageal cancer risk.  Vitamin D controls many downstream regulators of cellular processes including proliferation, apoptosis, and differentiation.  We evaluated the effects of vitamin D supplementation on global gene expression in patients with Barrett's esophagus.
Line 13: !Series_summary = We us


Gene annotation preview:
{'ID': [7896736, 7896738, 7896740, 7896742, 7896744], 'GB_LIST': [nan, nan, 'NM_001004195,NM_001005240,NM_001005484,BC136848,BC136867,BC136907,BC136908', 'NR_024437,XM_006711854,XM_006726377,XR_430662,AK298283,AL137655,BC032332,BC118988,BC122537,BC131690,NM_207366,AK301928,BC071667', 'NM_001005221,NM_001005224,NM_001005277,NM_001005504,BC137547,BC137568'], 'SPOT_ID': ['chr1:53049-54936', 'chr1:63015-63887', 'chr1:69091-70008', 'chr1:334129-334296', 'chr1:367659-368597'], 'seqname': ['chr1', 'chr1', 'chr1', 'chr1', 'chr1'], 'RANGE_GB': ['NC_000001.10', 'NC_000001.10', 'NC_000001.10', 'NC_000001.10', 'NC_000001.10'], 'RANGE_STRAND': ['+', '+', '+', '+', '+'], 'RANGE_START': ['53049', '63015', '69091', '334129', '367659'], 'RANGE_STOP': ['54936', '63887', '70008', '334296', '368597'], 'total_probes': [7, 31, 24, 6, 36], 'gene_assignment': ['---', 'ENST00000328113 // OR4G2P // olfactory receptor, family 4, subfamily G, member 2 pseudogene // --- // --- /// ENST000

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Determine which columns in the gene annotation data correspond to probe IDs and gene symbols
prob_col = 'ID'  # This is the column with the probe identifiers matching the gene expression data
gene_col = 'gene_assignment'  # This column contains gene symbol information

# First, let's examine what the gene_assignment column actually contains
print("Example gene assignment entries:")
for entry in gene_annotation[gene_col].head(3):
    print(f"- {entry[:100]}...")

# 2. Get the gene mapping dataframe by extracting these two columns
import re

# Let's define a better function to extract gene symbols from the gene_assignment field
def extract_gene_symbols_from_assignment(assignment_text):
    if not isinstance(assignment_text, str) or assignment_text == '---':
        return []
    
    # The format appears to be: ID // SYMBOL // DESCRIPTION // LOCATION // GENE_ID
    # We want to extract SYMBOL
    genes = []
    # Split by /// which separates multiple gene entries
    for entry in assignment_text.split('///'):
        # Apply regex to extract gene symbol between first and second //
        match = re.search(r'//\s+([A-Z0-9-]+)\s+//', entry)
        if match:
            gene_symbol = match.group(1)
            # Filter out common non-gene entries and limit to likely human gene symbols
            if (gene_symbol not in ['---', 'LOC'] and 
                not gene_symbol.startswith('LOC') and 
                re.match(r'^[A-Z][A-Z0-9-]{1,15}$', gene_symbol)):
                genes.append(gene_symbol)
    
    return genes

# Use a custom function when creating the mapping
mapping_data = gene_annotation[[prob_col, gene_col]].copy()
mapping_data = mapping_data.dropna()
mapping_data['Gene'] = mapping_data[gene_col].apply(extract_gene_symbols_from_assignment)
mapping_data = mapping_data[[prob_col, 'Gene']]
mapping_data = mapping_data.astype({prob_col: 'str'})

print(f"Gene mapping dataframe shape: {mapping_data.shape}")
print("First few rows of mapping data with extracted gene symbols:")
for idx, row in mapping_data.head(5).iterrows():
    print(f"ID: {row[prob_col]}, Genes: {row['Gene']}")

# Check if we have enough valid gene mappings
valid_mappings = mapping_data[mapping_data['Gene'].apply(len) > 0]
print(f"Number of probes with valid gene mappings: {len(valid_mappings)}")

# 3. Apply the gene mapping to convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, mapping_data)

print(f"Gene expression dataframe shape before normalization: {gene_data.shape}")
print("First few gene symbols in the gene expression data:")
print(gene_data.index[:10])

# Skip the normalization step for now as it's causing data loss
# Instead of: gene_data = normalize_gene_symbols_in_index(gene_data)

# Save the processed gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Example gene assignment entries:
- ---...
- ENST00000328113 // OR4G2P // olfactory receptor, family 4, subfamily G, member 2 pseudogene // --- /...
- NM_001004195 // OR4F4 // olfactory receptor, family 4, subfamily F, member 4 // 15q26.3 // 26682 ///...


Gene mapping dataframe shape: (33297, 2)
First few rows of mapping data with extracted gene symbols:
ID: 7896736, Genes: []
ID: 7896738, Genes: ['OR4G2P', 'OR4G11P', 'OR4G1P']
ID: 7896740, Genes: ['OR4F4', 'OR4F17', 'OR4F5', 'OR4F17', 'OR4F4', 'OR4F5', 'OR4F17', 'OR4F17', 'OR4F17', 'OR4F4', 'OR4F4']
ID: 7896742, Genes: ['LINC00266-1', 'LINC00266-3', 'LINC00266-1', 'PCMTD2', 'LINC00266-1', 'LINC00266-1', 'SEPT14', 'SEPT14', 'LINC00266-4P', 'LINC00266-2P', 'SEPT14']
ID: 7896744, Genes: ['OR4F29', 'OR4F3', 'OR4F16', 'OR4F21', 'OR4F21', 'OR4F3', 'OR4F16', 'OR4F29', 'OR4F7P', 'OR4F1P', 'OR4F2P', 'OR4F3', 'OR4F16', 'OR4F29', 'OR4F3', 'OR4F16', 'OR4F29', 'OR4F28P', 'OR4F3', 'OR4F16', 'OR4F29', 'OR4F3', 'OR4F16', 'OR4F29', 'OR4F8P']
Number of probes with valid gene mappings: 24436
Gene expression dataframe shape before normalization: (0, 76)
First few gene symbols in the gene expression data:
Index([], dtype='object', name='Gene')
Gene expression data saved to ../../output/preprocess/Esophagea

### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Normalized gene data shape: {normalized_gene_data.shape}")
print("First few genes with their expression values after normalization:")
print(normalized_gene_data.head())

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Check if trait data is available before proceeding with clinical data extraction
if trait_row is None:
    print("Trait row is None. Cannot extract trait information from clinical data.")
    # Create an empty dataframe for clinical features
    clinical_features = pd.DataFrame()
    
    # Create an empty dataframe for linked data
    linked_data = pd.DataFrame()
    
    # Validate and save cohort info
    validate_and_save_cohort_info(
        is_final=True, 
        cohort=cohort, 
        info_path=json_path, 
        is_gene_available=True, 
        is_trait_available=False,  # Trait data is not available
        is_biased=True,  # Not applicable but required
        df=pd.DataFrame(),  # Empty dataframe
        note="Dataset contains gene expression data but lacks clear trait indicators for Duchenne Muscular Dystrophy status."
    )
    print("Data was determined to be unusable due to missing trait indicators and was not saved")
else:
    try:
        # Get the file paths for the matrix file to extract clinical data
        _, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
        
        # Get raw clinical data from the matrix file
        _, clinical_raw = get_background_and_clinical_data(matrix_file)
        
        # Verify clinical data structure
        print("Raw clinical data shape:", clinical_raw.shape)
        
        # Extract clinical features using the defined conversion functions
        clinical_features = geo_select_clinical_features(
            clinical_df=clinical_raw,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        print("Clinical features:")
        print(clinical_features)
        
        # Save clinical features to file
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        clinical_features.to_csv(out_clinical_data_file)
        print(f"Clinical features saved to {out_clinical_data_file}")
        
        # 3. Link clinical and genetic data
        linked_data = geo_link_clinical_genetic_data(clinical_features, normalized_gene_data)
        print(f"Linked data shape: {linked_data.shape}")
        print("Linked data preview (first 5 rows, first 5 columns):")
        print(linked_data.iloc[:5, :5])
        
        # 4. Handle missing values
        print("Missing values before handling:")
        print(f"  Trait ({trait}) missing: {linked_data[trait].isna().sum()} out of {len(linked_data)}")
        if 'Age' in linked_data.columns:
            print(f"  Age missing: {linked_data['Age'].isna().sum()} out of {len(linked_data)}")
        if 'Gender' in linked_data.columns:
            print(f"  Gender missing: {linked_data['Gender'].isna().sum()} out of {len(linked_data)}")
        
        gene_cols = [col for col in linked_data.columns if col not in [trait, 'Age', 'Gender']]
        print(f"  Genes with >20% missing: {sum(linked_data[gene_cols].isna().mean() > 0.2)}")
        print(f"  Samples with >5% missing genes: {sum(linked_data[gene_cols].isna().mean(axis=1) > 0.05)}")
        
        cleaned_data = handle_missing_values(linked_data, trait)
        print(f"Data shape after handling missing values: {cleaned_data.shape}")
        
        # 5. Evaluate bias in trait and demographic features
        is_trait_biased = False
        if len(cleaned_data) > 0:
            trait_biased, cleaned_data = judge_and_remove_biased_features(cleaned_data, trait)
            is_trait_biased = trait_biased
        else:
            print("No data remains after handling missing values.")
            is_trait_biased = True
        
        # 6. Final validation and save
        is_usable = validate_and_save_cohort_info(
            is_final=True, 
            cohort=cohort, 
            info_path=json_path, 
            is_gene_available=True, 
            is_trait_available=True, 
            is_biased=is_trait_biased, 
            df=cleaned_data,
            note="Dataset contains gene expression data comparing Duchenne muscular dystrophy vs healthy samples."
        )
        
        # 7. Save if usable
        if is_usable and len(cleaned_data) > 0:
            os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
            cleaned_data.to_csv(out_data_file)
            print(f"Linked data saved to {out_data_file}")
        else:
            print("Data was determined to be unusable or empty and was not saved")
            
    except Exception as e:
        print(f"Error processing data: {e}")
        # Handle the error case by still recording cohort info
        validate_and_save_cohort_info(
            is_final=True, 
            cohort=cohort, 
            info_path=json_path, 
            is_gene_available=True, 
            is_trait_available=False,  # Mark as not available due to processing issues
            is_biased=True, 
            df=pd.DataFrame(),  # Empty dataframe
            note=f"Error processing data: {str(e)}"
        )
        print("Data was determined to be unusable and was not saved")

Normalized gene data shape: (0, 76)
First few genes with their expression values after normalization:
Empty DataFrame
Columns: [GSM2694849, GSM2694850, GSM2694851, GSM2694852, GSM2694853, GSM2694854, GSM2694855, GSM2694856, GSM2694857, GSM2694858, GSM2694859, GSM2694860, GSM2694861, GSM2694862, GSM2694863, GSM2694864, GSM2694865, GSM2694866, GSM2694867, GSM2694868, GSM2694869, GSM2694870, GSM2694871, GSM2694872, GSM2694873, GSM2694874, GSM2694875, GSM2694876, GSM2694877, GSM2694878, GSM2694879, GSM2694880, GSM2694881, GSM2694882, GSM2694883, GSM2694884, GSM2694885, GSM2694886, GSM2694887, GSM2694888, GSM2694889, GSM2694890, GSM2694891, GSM2694892, GSM2694893, GSM2694894, GSM2694895, GSM2694896, GSM2694897, GSM2694898, GSM2694899, GSM2694900, GSM2694901, GSM2694902, GSM2694903, GSM2694904, GSM2694905, GSM2694906, GSM2694907, GSM2694908, GSM2694909, GSM2694910, GSM2694911, GSM2694912, GSM2694913, GSM2694914, GSM2694915, GSM2694916, GSM2694917, GSM2694918, GSM2694919, GSM2694920, GSM26949

Raw clinical data shape: (3, 77)
Clinical features:
                   GSM2694849  GSM2694850  GSM2694851  GSM2694852  GSM2694853  \
Esophageal_Cancer         1.0         1.0         0.0         0.0         1.0   

                   GSM2694854  GSM2694855  GSM2694856  GSM2694857  GSM2694858  \
Esophageal_Cancer         0.0         1.0         0.0         1.0         0.0   

                   ...  GSM2694915  GSM2694916  GSM2694917  GSM2694918  \
Esophageal_Cancer  ...         1.0         0.0         1.0         0.0   

                   GSM2694919  GSM2694920  GSM2694921  GSM2694922  GSM2694923  \
Esophageal_Cancer         1.0         0.0         1.0         0.0         1.0   

                   GSM2694924  
Esophageal_Cancer         0.0  

[1 rows x 76 columns]
Clinical features saved to ../../output/preprocess/Esophageal_Cancer/clinical_data/GSE100843.csv
Linked data shape: (76, 1)
Linked data preview (first 5 rows, first 5 columns):
            Esophageal_Cancer
GSM2694849      